IMPORTS PARA O PROGRAMA E NORMALIZAÇÃO DO TITLE


In [ ]:
import pandas as pd
import re
df = pd.read_csv('data.csv')
if df['title'].dtype == 'object':
    df['title'] = df['title'].str.upper()
dfp = df[df['amount'] > 0]
print(dfp)
df_cf = pd.read_csv('categorias_financeiras_organizado.csv')
print(df_cf)
df_usr = pd.read_csv('categorias_financeiras_user.csv')
print(df_usr)
vl = pd.read_csv('valuesmax.csv')
print(vl)
#IMPORTACAO DO PANDAS E DAS LISTAS PYTHON

CRIACAO DO PRIMEIRO MODULO (ADICAO DE FILTROS - INICIALMENTE PRE DEFINIDOS - E ESTIPULACAO DE METAS PARA GASTOS)

In [ ]:
def adicionar_categoria_gastos():
    nome = input('Digite o nome do centro de custo desejado')
    titulo = input('Digite a palavra chave que identifica o titulo do gasto no extrato *EX: Corrida do Uber = Uber*')
    nome_padrao = nome.upper().lstrip().rstrip()
    titulo_padrao = titulo.upper().lstrip().rstrip()
    save = 'BOTAO SALVAR'
    cancel = 'BOTAO CANCELAR'

    if df_cf['CATEGORIA'].isin([nome_padrao]).any() or df_usr['CATEGORIA'].isin([nome_padrao]).any() :
        print('Essa categoria ja existe! Voce pode adicionar palavras chave para que seu gasto entre nas categorias padroes')
    elif nome == "" or titulo == "":
        print('Nao é possivel adicionar valores vazios')
    elif nome == "" and titulo == "":
        print('Nao é possivel adicionar valores vazios')
    else:
        nova_categoria = df_usr.loc[len(df_usr)] = [nome_padrao,titulo_padrao]
        print(nova_categoria)
        print(df_usr)
        df_usr.to_csv('categorias_financeiras_user.csv', index=False)

adicionar_categoria_gastos()


FILTRANDO CATEGORIAS (PRINTA FATURA SOMENTE COM AS CATEGORIAS PADRAO)

In [ ]:
def filtro_categorias_default():
    filtros_padrao = df_cf['TITULO']
    regex_filtros_padrao = '|'.join(filtros_padrao.dropna().astype(str).map(re.escape))
    filtros_padroes = dfp[dfp['title'].str.contains(regex_filtros_padrao, case=False, na=False)]
    soma = (filtros_padroes[filtros_padroes['amount'] > 0]['amount'].sum())
    print(filtros_padroes)
    print(soma)

filtro_categorias_default()

FILTRANDO GASTOS GERAIS (RETIRA OS GASTOS FILTRADOS E MOSTRA OS "SEM CATEGORIA")

In [ ]:
def filtro_geral():
    filtros_padrao = df_cf['TITULO']
    regex_filtros_padrao = '|'.join(filtros_padrao.dropna().astype(str).map(re.escape))
    filtros_padroes = dfp[~(dfp['title'].str.contains(regex_filtros_padrao, case=False, na=False))]
    soma = (filtros_padroes[filtros_padroes['amount'] > 0]['amount'].sum())
    print(filtros_padroes)
    print(soma)

filtro_geral()

BUSCANDO APENAS A CATEGORIA ESCOLHIDA

In [ ]:
def pesquisar_categoria():
    categoria = input('Digite a categoria: ').strip().upper()
    regra = df_cf[df_cf['CATEGORIA'].str.upper() == categoria]
    regra_usr = df_usr[df_usr['CATEGORIA'].str.upper() == categoria]
    if regra.empty and regra_usr.empty:
        print('CATEGORIA NAO ENCONTRADA')
        return
    padroes = []
    if not regra.empty:
        padroes.extend(regra['TITULO'].dropna().astype(str))
    if not regra_usr.empty:
        padroes.extend(regra_usr['TITULO'].dropna().astype(str))
    regex = '|'.join(map(re.escape, padroes))
    resultado = df[df['title'].str.contains(regex, case=False, na=False)]
    soma = resultado['amount'].sum()
    print(resultado)
    print(soma)
pesquisar_categoria()

INSERINDO PALAVRAS CHAVE EM CATEGORIAS EXISTENTES

In [ ]:
def palavras_chave_categorias_dflt():
    categorias = input('Digite a categoria existente desejada:')
    categorias_padrao = categorias.upper().lstrip().rstrip()
    if df_cf['CATEGORIA'].isin([categorias_padrao]).any():
        titulo = input('Digite as palavras-chave que deseja incluir para reconhecimento do extrato')
        titulo_padrao = titulo.upper().lstrip().rstrip()
        nova_plchave = df_cf.loc[len(df_cf)] = [categorias_padrao,titulo_padrao]
        print(nova_plchave)
    else:
        print('Essa categoria nao existe! Voce pode criá-la ou incluir em alguma das categorias existentes')

palavras_chave_categorias_dflt()

ESTIPULANDO VALOR MAXIMO PARA CADA CATEGORIA

In [ ]:
def valor_maximo_categoria():
    while True:
        categorias = input('Digite a categoria que deseja configurar:')
        categorias_padrao = categorias.upper().lstrip().rstrip()
        if df_cf['CATEGORIA'].isin([categorias_padrao]).any() or df_usr['CATEGORIA'].isin([categorias_padrao]).any() :
            valor = float(input('Digite o valor maximo que deseja gastar:'))
        else:
            print('Essa categoria nao existe!')
            return False
        
        if vl['CATEGORIA'].isin([categorias_padrao]).any():
            print('Voce ja adicionou um valor para essa categoria')
            return False 
        add_valor = vl.loc[len(vl)] = [categorias_padrao,valor]
        print(add_valor)
        vl.to_csv('valuesmax.csv', index=False)
        return False

valor_maximo_categoria()

VERIFICADOR DE VALOR POR CATEGORIA

In [107]:
def total_por_categoria():
    totais = {}

    categorias = pd.concat([
        df_cf['CATEGORIA'],
        df_usr['CATEGORIA']
    ]).dropna().unique()

    for categoria in categorias:
        regras = df_cf[df_cf['CATEGORIA'] == categoria]
        regras_usr = df_usr[df_usr['CATEGORIA'] == categoria]

        padroes = []

        if not regras.empty:
            padroes.extend(regras['TITULO'].dropna().astype(str))

        if not regras_usr.empty:
            padroes.extend(regras_usr['TITULO'].dropna().astype(str))

        if not padroes:
            continue

        regex = '|'.join(map(re.escape, padroes))

        resultado = df[df['title'].str.contains(regex, case=False, na=False)]

        total_gasto = resultado['amount'].sum()

        vl_cat = vl[vl['CATEGORIA'] == categoria]

        if not vl_cat.empty:
            valor_limite = vl_cat['VALOR'].iloc[0]
            percentual = (total_gasto / valor_limite * 100) if valor_limite > 0 else 0
        else:
            valor_limite = None
            percentual = None

        totais[categoria.upper()] = {
            'total': total_gasto,
            'limite': valor_limite,
            'percentual': percentual
        }

    for cat, dados in totais.items():
        total = dados['total']
        limite = dados['limite']
        perc = dados['percentual']

        if limite is not None:
            print(f'{cat} - {total:.2f} / {limite:.2f} ({perc:.1f}%)'.upper())
        else:
            print(f'{cat} - {total:.2f} (SEM VALOR ESTIPULADO)'.upper())

total_por_categoria()

DELIVERY - 114.35 / 500.00 (22.9%)
VAREJO - 186.15 (SEM VALOR ESTIPULADO)
TRANSPORTE - 19.90 (SEM VALOR ESTIPULADO)
MERCADO - 0.00 / 1000.00 (0.0%)
ENTRETENIMENTO - 0.00 / 250.00 (0.0%)
SAUDE - 0.00 / 500.00 (0.0%)
CONTAS - 0.00 (SEM VALOR ESTIPULADO)
ASSINATURA - 39.90 / 50.00 (79.8%)
GABINETE - 0.00 (SEM VALOR ESTIPULADO)


RETIRANDO CATEGORIA DO USUARIO

In [ ]:
def retirar_categoria():
    global df_usr
    while True:
        print(df_usr)
        categoria_usr = input('Qual categoria deseja retirar?').upper().lstrip().rstrip()
        if df_usr['CATEGORIA'].isin([categoria_usr]).any():
            print(df_usr[df_usr['CATEGORIA'] == categoria_usr])
        else:
            print('Essa categoria nao existe!')
            return False
        titulo = input('Digite os titulos que voce deseja remover (caso queira remover toda a categoria, digite TODOS)').upper().lstrip().rstrip()
        if df_usr['TITULO'].isin([titulo]).any():
            index = (df_usr[df_usr['TITULO'] == titulo]).index
            df_usr = df_usr.drop(index)
            df_usr.to_csv('categorias_financeiras_user.csv', index=False)
            print(df_usr)
            return False
        else:
            print('Esse titulo nao existe')
            return False
retirar_categoria()

In [ ]:
print(dfp)

In [ ]:
#COMO EU VOU SOMAR OS GASTOS POR FILTRO, PARA NO FINAL TIRAR A PORCENTAGEM EM RELACAO AO QUANTO EU GASTEI DESSAS CATEGORIAS COM O TOTAL QUE EU ESTIPULEI

'''
EXEMPLO:

COM DELIVERY, GASTEI 380, MAS O TOTAL QUE EU ESTIPULEI FOI DE 500 REAIS

-PRECISO PEGAR DO EXTRATO TODOS OS GASTOS QUE SE ENQUADRAM COM DELIVERY
-SOMAR A COLUNA DE AMOUNT
-DIVIDIR ESSE VALOR COM O TOTAL E MULTIPLICAR POR 100
-EM CIMA DO VALOR GERADO (PORCENTAGEM), UTILIZAR ESSE VALOR PRA INTERAGIR COM O FRONT E FAZER UM GRAFICO (DE 0-30 VERDE, DE 30-60 AMARELO, DE 60-100 VERMELHO)

'''